In [1]:
import pandas as pd
import lightgbm as lgb
import time

In [2]:
def NDVI(row):
    ndvi = (row['08NIR'] - row['04Red']) / (row['08NIR'] + row['04Red'])
    return ndvi
def EVI(row):
    evi  = (row['08NIR'] - row['04Red'])/((row['08NIR']+6*row['04Red']-7.5*row['02Blue'])+1)
    return evi
def CVI(row):
    cvi = row['08NIR']*(row['04Red']/(row['03Green'])**2)
    return cvi
def NBR(row):
    nbr = (row['08NIR'] - row['13SWIR2'])/(row['08NIR'] + row['13SWIR2'])
    return nbr
def ARI(row):
    ari = (1/row['03Green'])-(1/row['04Red'])
    return ari
def TVI(row):
    tvi = 0.5 * (120 * (row['08NIR'] - row['03Green']) - 200 * (row['04Red'] - row['03Green']))
    return tvi
def VARI(row):
    vari = (row['03Green']-row['04Red'])/(row['03Green']+row['04Red']-row['02Blue'])
    return vari
def NDRE(row):
    ndre = (row['08NIR'] - row['05Veg5']) / (row['08NIR'] + row['05Veg5'])
    return ndre
def RDVI(row):
    rdvi = (row['08NIR'] - row['04Red']) / ((row['08NIR'] + row['04Red']) ** 0.5)
    return rdvi
def GLI(row):
    gli = (2 * row['03Green'] - row['04Red'] - row['02Blue']) / (2 * row['03Green'] + row['04Red'] + row['02Blue'])
    return gli

In [3]:
# test.csv
train_df = pd.read_csv("/kaggle/input/forest-type-classification-spai/train.csv")
# train_df = train_df[0:10]
# train_df
train_df = train_df.rename(columns = {
    'b1':'01Coastal',
    'b2':'02Blue',
    'b3':'03Green',
    'b4':'04Red',
    'b5':'05Veg5',
    'b6':'06Veg6',
    'b7':'07Veg7',
    'b8':'08NIR',
    'b8_a':'09NIRn',
    'b9':'10WV',
    'b10':'11SWIR-C',
    'b11':'12SWIR1',
    'b12':'13SWIR2'
})
train_df

,id,01Coastal,12SWIR1,13SWIR2,02Blue,03Green,04Red,05Veg5,06Veg6,07Veg7,08NIR,09NIRn,10WV,nforest_type
0,2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,MDF
1,3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,DDF
2,13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,MDF
3,17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,MDF
4,5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,MDF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,374,1940,1054,382,565,498,977,1678,1929,2109,2291,2100,DDF
13049,13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,DDF
13050,755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,DDF
13051,1616,1174,2312,1190,1112,1126,889,1310,2511,3085,3050,3396,3380,MDF


In [4]:
train_df['NDVI'] = train_df.apply(NDVI, axis=1)
train_df['EVI']  = train_df.apply(EVI, axis=1)
train_df['CVI']  = train_df.apply(CVI, axis=1)
train_df['NBR']  = train_df.apply(NBR, axis=1)
train_df['ARI']  = train_df.apply(ARI, axis=1)
train_df['TVI']  = train_df.apply(TVI, axis=1)
train_df['VARI'] = train_df.apply(VARI, axis=1)
train_df['NDRE'] = train_df.apply(NDRE, axis=1)
train_df['RDVI'] = train_df.apply(VARI, axis=1)
train_df['GLI']  = train_df.apply(NDRE, axis=1)
train_df

,id,01Coastal,12SWIR1,13SWIR2,02Blue,03Green,04Red,05Veg5,06Veg6,07Veg7,...,NDVI,EVI,CVI,NBR,ARI,TVI,VARI,NDRE,RDVI,GLI
0,2002,293,1927,1038,278,475,453,987,1773,2184,...,0.614960,0.571034,3.814737,0.293397,-0.000102,87700.0,0.033846,0.316245,0.033846,0.316245
1,3212,197,1598,697,201,347,228,682,1982,2449,...,0.816277,0.957693,4.268053,0.527618,-0.001504,126320.0,0.318182,0.535422,0.318182,0.535422
2,13312,929,1975,1031,982,1020,856,1220,2051,2421,...,0.472906,9.365854,1.968043,0.397604,-0.000188,98720.0,0.183445,0.324474,0.183445,0.324474
3,17020,132,1560,689,189,408,175,609,2117,2907,...,0.890591,1.072060,3.179066,0.628872,-0.003263,180260.0,0.591371,0.664740,0.591371,0.664740
4,5967,241,1944,1131,362,538,487,918,1549,1844,...,0.555048,0.636126,2.863677,0.201553,-0.000195,74940.0,0.076923,0.299237,0.076923,0.299237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,374,1940,1054,382,565,498,977,1678,1929,...,0.617952,0.721451,3.290099,0.333544,-0.000238,99340.0,0.098385,0.366818,0.098385,0.366818
13049,13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,...,0.379262,0.915959,2.182249,0.181216,-0.000005,130120.0,0.008307,0.258096,0.008307,0.258096
13050,755,940,2007,1148,975,1080,968,1252,1780,1983,...,0.334708,2.221209,1.611674,0.256958,-0.000107,62920.0,0.104380,0.216030,0.104380,0.216030
13051,1616,1174,2312,1190,1112,1126,889,1310,2511,3085,...,0.548616,48.022222,2.138577,0.438679,-0.000237,139140.0,0.262458,0.399083,0.262458,0.399083


In [5]:
# Important_11_feature = ['id','12SWIR1','10WV','13SWIR2','09NIRn','01Coastal','05Veg5','CVI','NDRE','NBR','02Blue',
# 'nforest_type']

In [6]:
# train_df = train_df[Important_11_feature]
train_df

,id,01Coastal,12SWIR1,13SWIR2,02Blue,03Green,04Red,05Veg5,06Veg6,07Veg7,...,NDVI,EVI,CVI,NBR,ARI,TVI,VARI,NDRE,RDVI,GLI
0,2002,293,1927,1038,278,475,453,987,1773,2184,...,0.614960,0.571034,3.814737,0.293397,-0.000102,87700.0,0.033846,0.316245,0.033846,0.316245
1,3212,197,1598,697,201,347,228,682,1982,2449,...,0.816277,0.957693,4.268053,0.527618,-0.001504,126320.0,0.318182,0.535422,0.318182,0.535422
2,13312,929,1975,1031,982,1020,856,1220,2051,2421,...,0.472906,9.365854,1.968043,0.397604,-0.000188,98720.0,0.183445,0.324474,0.183445,0.324474
3,17020,132,1560,689,189,408,175,609,2117,2907,...,0.890591,1.072060,3.179066,0.628872,-0.003263,180260.0,0.591371,0.664740,0.591371,0.664740
4,5967,241,1944,1131,362,538,487,918,1549,1844,...,0.555048,0.636126,2.863677,0.201553,-0.000195,74940.0,0.076923,0.299237,0.076923,0.299237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,374,1940,1054,382,565,498,977,1678,1929,...,0.617952,0.721451,3.290099,0.333544,-0.000238,99340.0,0.098385,0.366818,0.098385,0.366818
13049,13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,...,0.379262,0.915959,2.182249,0.181216,-0.000005,130120.0,0.008307,0.258096,0.008307,0.258096
13050,755,940,2007,1148,975,1080,968,1252,1780,1983,...,0.334708,2.221209,1.611674,0.256958,-0.000107,62920.0,0.104380,0.216030,0.104380,0.216030
13051,1616,1174,2312,1190,1112,1126,889,1310,2511,3085,...,0.548616,48.022222,2.138577,0.438679,-0.000237,139140.0,0.262458,0.399083,0.262458,0.399083


In [7]:
test_df = pd.read_csv("/kaggle/input/forest-type-classification-spai/test.csv")
# test_df = test_df[0:10]
test_df = test_df.rename(columns = {
    'b1':'01Coastal',
    'b2':'02Blue',
    'b3':'03Green',
    'b4':'04Red',
    'b5':'05Veg5',
    'b6':'06Veg6',
    'b7':'07Veg7',
    'b8':'08NIR',
    'b8_a':'09NIRn',
    'b9':'10WV',
    'b10':'11SWIR-C',
    'b11':'12SWIR1',
    'b12':'13SWIR2'
})
test_df['NDVI'] = test_df.apply(NDVI, axis=1)
test_df['EVI']  = test_df.apply(EVI, axis=1)
test_df['CVI']  = test_df.apply(CVI, axis=1)
test_df['NBR']  = test_df.apply(NBR, axis=1)
test_df['ARI']  = test_df.apply(ARI, axis=1)
test_df['TVI']  = test_df.apply(TVI, axis=1)
test_df['VARI'] = test_df.apply(VARI, axis=1)
test_df['NDRE'] = test_df.apply(NDRE, axis=1)
test_df['RDVI'] = test_df.apply(VARI, axis=1)
test_df['GLI']  = test_df.apply(NDRE, axis=1)
test_df

,id,01Coastal,12SWIR1,13SWIR2,02Blue,03Green,04Red,05Veg5,06Veg6,07Veg7,...,NDVI,EVI,CVI,NBR,ARI,TVI,VARI,NDRE,RDVI,GLI
0,13467,69,1425,693,312,524,376,847,1821,2356,...,0.726943,0.872331,3.256395,0.548681,-0.000751,126040.0,0.251701,0.474729,0.251701,0.474729
1,12719,242,1514,691,343,522,324,718,1730,2178,...,0.768240,1.164543,2.939358,0.563073,-0.001171,136800.0,0.393638,0.549843,0.393638,0.549843
2,1054,218,2354,1118,292,596,410,965,2586,3226,...,0.783126,0.813015,3.890900,0.501894,-0.000761,185100.0,0.260504,0.554889,0.260504,0.554889
3,13747,350,2013,1134,306,572,475,982,1754,1935,...,0.654545,0.635818,3.302805,0.334702,-0.000357,111880.0,0.130904,0.396991,0.130904,0.396991
4,9453,185,1450,712,293,440,384,673,1487,1965,...,0.704274,0.788192,4.389421,0.513162,-0.000331,111980.0,0.105461,0.533611,0.105461,0.533611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,115,447,1686,811,425,661,441,958,2432,2891,...,0.741121,1.041022,2.993690,0.570559,-0.000755,160300.0,0.324963,0.511723,0.324963,0.511723
3996,10654,252,2694,1503,470,778,753,1294,2334,2656,...,0.561189,0.524367,3.332794,0.281205,-0.000043,116560.0,0.023563,0.348603,0.023563,0.348603
3997,5718,233,1486,618,249,409,260,699,2188,2831,...,0.841945,1.017074,4.709441,0.661184,-0.001401,172160.0,0.354762,0.625101,0.354762,0.625101
3998,13054,221,1840,774,245,441,231,703,2491,3453,...,0.868563,1.077466,3.900659,0.618531,-0.002061,191580.0,0.491803,0.647354,0.491803,0.647354


In [8]:
# Important_10_feature = ['id','12SWIR1','10WV','13SWIR2','09NIRn','01Coastal','05Veg5','CVI','NDRE','NBR','02Blue']
# test_df = test_df[Important_10_feature]
test_df

,id,01Coastal,12SWIR1,13SWIR2,02Blue,03Green,04Red,05Veg5,06Veg6,07Veg7,...,NDVI,EVI,CVI,NBR,ARI,TVI,VARI,NDRE,RDVI,GLI
0,13467,69,1425,693,312,524,376,847,1821,2356,...,0.726943,0.872331,3.256395,0.548681,-0.000751,126040.0,0.251701,0.474729,0.251701,0.474729
1,12719,242,1514,691,343,522,324,718,1730,2178,...,0.768240,1.164543,2.939358,0.563073,-0.001171,136800.0,0.393638,0.549843,0.393638,0.549843
2,1054,218,2354,1118,292,596,410,965,2586,3226,...,0.783126,0.813015,3.890900,0.501894,-0.000761,185100.0,0.260504,0.554889,0.260504,0.554889
3,13747,350,2013,1134,306,572,475,982,1754,1935,...,0.654545,0.635818,3.302805,0.334702,-0.000357,111880.0,0.130904,0.396991,0.130904,0.396991
4,9453,185,1450,712,293,440,384,673,1487,1965,...,0.704274,0.788192,4.389421,0.513162,-0.000331,111980.0,0.105461,0.533611,0.105461,0.533611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,115,447,1686,811,425,661,441,958,2432,2891,...,0.741121,1.041022,2.993690,0.570559,-0.000755,160300.0,0.324963,0.511723,0.324963,0.511723
3996,10654,252,2694,1503,470,778,753,1294,2334,2656,...,0.561189,0.524367,3.332794,0.281205,-0.000043,116560.0,0.023563,0.348603,0.023563,0.348603
3997,5718,233,1486,618,249,409,260,699,2188,2831,...,0.841945,1.017074,4.709441,0.661184,-0.001401,172160.0,0.354762,0.625101,0.354762,0.625101
3998,13054,221,1840,774,245,441,231,703,2491,3453,...,0.868563,1.077466,3.900659,0.618531,-0.002061,191580.0,0.491803,0.647354,0.491803,0.647354


In [9]:
print("train data shape:",train_df.shape)
print("test data shape :",test_df.shape)

train data shape: (13053, 24)
test data shape : (4000, 23)


<hr>

In [10]:
X = train_df.drop(columns=['nforest_type','id'])
y = train_df['nforest_type']
X_test  = test_df

In [11]:
# clf = lgb.LGBMClassifier()
# clf.fit(X_train, y_train)

In [12]:
import optuna
from sklearn.model_selection import cross_val_score

In [ ]:
start = time.time()
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_leaves': trial.suggest_int('num_leaves', 31, 2047),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'device_type': 'gpu',  # Ensure GPU is used if required
        # Additional parameters
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),  # L2 regularization
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0)  # Useful for imbalanced datasets
    }
    
    clf = lgb.LGBMClassifier(
        **params
    )
    
    return cross_val_score(
        clf, X, y, n_jobs=-1, cv=5
    ).mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
end   = time.time()


[I 2024-06-04 02:24:38,829] A new study created in memory with name: no-name-0d2ba283-063a-4925-a56c-75ac76204294


[LightGBM] [Warning] feature_fraction is set=0.8882142184488808, colsample_bytree=0.7949391219214395 will be ignored. Current value: feature_fraction=0.8882142184488808
[LightGBM] [Warning] lambda_l2 is set=0.004655596398511932, reg_lambda=4.1123329456992754e-07 will be ignored. Current value: lambda_l2=0.004655596398511932
[LightGBM] [Warning] lambda_l1 is set=0.0018810954450208038, reg_alpha=2.063456006430896e-05 will be ignored. Current value: lambda_l1=0.0018810954450208038
[LightGBM] [Warning] bagging_fraction is set=0.715448940257414, subsample=0.5327250493570559 will be ignored. Current value: bagging_fraction=0.715448940257414
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] feature_fractio

In [ ]:
print(end-start)
trial = study.best_trial
print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
best_params = trial.params
best_params

In [ ]:
final_clf = lgb.LGBMClassifier(
    **best_params,
    device_type = 'GPU'
)
final_clf.fit(X,y)

In [ ]:
# y_pred = final_clf.predict(X_test)
# y_pred
final_clf.predict(X_test[X_test['id'] == 13467].drop(columns=['id']))

In [ ]:
df = pd.read_csv("/kaggle/input/forest-type-classification-spai/sample_submission.csv")
for idx,row in df.iterrows():
#     print(X_test[X_test['id'] == row['id']])
    data = X_test[X_test['id'] == row['id']].drop(columns=['id'])      # เปิดข้อมูลรายตัวตาม id ของ sample_submission.csv file
    result = final_clf.predict(data)
    df.at[idx,'nforest_type'] = result
#     break

In [ ]:
df

In [ ]:
version = '0.1.7'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
df.to_csv(csv_name,index=False)

<hr>

- ค่า reflextant ของพืช
- phyiological status of plant